In [1]:
import tsl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric

from torch.optim import Adam
from tsl.datasets import PeMS04, PeMS07, PeMS08, PemsBay
from tsl.datasets import MetrLA
from tsl.data import SpatioTemporalDataset
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
from tsl.data.preprocessing import StandardScaler
from einops.layers.torch import Rearrange

import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
dataset_MetrLA = MetrLA(root='data/MetrLA')

# get_connectivity uses get_similarity under the hood
connectivity = dataset_MetrLA.get_connectivity(threshold=0.1, include_self=False, normalize_axis=1, layout="edge_index")

# subclass of torch.utils.data.Dataset
torch_dataset = SpatioTemporalDataset(
    target=dataset_MetrLA.dataframe(),
    connectivity=connectivity,
    mask=dataset_MetrLA.mask,
    horizon=6,
    window=18,
    stride=1
)

scalers = {'target': StandardScaler(axis=(0, 1))}

# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=0.1, test_len=0.2)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
)

dm.setup()

train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/tsl/datasets/metr_la.py:98: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(df.index[0], df.index[-1], freq='5T')
/users/hunjael/.conda/envs/torch-st/lib/python3.10/site-packages/tsl/datasets/metr_la.py:109: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  df = df.replace(to_replace=0., method='ffill')


### Modeling Attention Schemes with ST data

In [3]:
# feature vectorization approach (flatten the temporal dim along the feature dim)
class SelfAttention(nn.Module):
    def __init__(self, input_size, hidden_size, window_size, horizon, num_heads=8, dropout=0.6):
        super(SelfAttention, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.window_size = window_size
        self.horizon = horizon
        self.num_heads = num_heads
        
        self.query = nn.Linear(input_size * window_size, hidden_size * num_heads)
        self.key = nn.Linear(input_size * window_size, hidden_size * num_heads)
        self.value = nn.Linear(input_size * window_size, hidden_size * num_heads)
        self.out = nn.Linear(hidden_size * num_heads, hidden_size)
        
        self.dropout = nn.Dropout(dropout)
        
        # STGNN specific utility functions
        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        
        self.scale = self.hidden_size ** 0.5

    def forward(self, x):
        # feature vectorization: flatten the temporal dim along the feature dim
        b, t, n, f = x.size()
        x = x.permute(0, 2, 1, 3).reshape(b, n, t * f)

        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        # Reshape for multi-head attention
        Q = Q.view(b, n, self.num_heads, self.hidden_size).transpose(1, 2)
        K = K.view(b, n, self.num_heads, self.hidden_size).transpose(1, 2)
        V = V.view(b, n, self.num_heads, self.hidden_size).transpose(1, 2)
        
        # e = torch.matmul(Q, K.transpose(-2, -1)) / self.scale
        e = (Q @ K.mT) / self.scale
        attention = F.softmax(e, dim=-1)
        
        # Apply dropout to attention weights
        attention = self.dropout(attention)
        
        out = attention @ V
        
        # combine the heads
        out = out.transpose(1, 2).contiguous().view(b, n, self.hidden_size * self.num_heads)
        out = self.out(out)
        
        out_horizon = self.decoder(out)
        out_horizon = out_horizon.view(b, n, self.horizon, f).permute(0, 2, 1, 3)
        
        return out_horizon
    

In [33]:
attention_block = SelfAttention(input_size=1, hidden_size=4, window_size=18, horizon=6, num_heads=8, dropout=0.6)

epochs = 1
criterion = nn.MSELoss()
optimizer = Adam(attention_block.parameters(), lr=0.001)

for epoch in range(epochs):
    attention_block.train()
    for batch in train_loader:
        x, edge_index, edge_weight, y = batch.x, batch.edge_index, batch.edge_weight, batch.y
        
        out = attention_block(x)
        loss = criterion(out, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}/{epochs} Loss: {loss.item()}')

eval_loss = 0
attention_block.eval()
with torch.no_grad():
    for batch in test_loader:
        x, edge_index, edge_weight, y = batch.x, batch.edge_index, batch.edge_weight, batch.y

        out = attention_block(x)
        eval_loss += criterion(out, y).item()

print(f'Evaluation Loss: {eval_loss / len(test_loader)}')
        
        

Epoch 1/1 Loss: 173.25393676757812
Evaluation Loss: 112.88236644532945


In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, input_size, hidden_size, window_size, horizon, num_heads=8, dropout=0.6):
        super(CrossAttention, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.window_size = window_size
        self.horizon = horizon
        self.num_heads = num_heads
        
        self.query = nn.Linear(input_size * window_size, hidden_size * num_heads)
        self.key = nn.Linear(input_size * window_size, hidden_size * num_heads)
        self.value = nn.Linear(input_size * window_size, hidden_size * num_heads)
        self.out = nn.Linear(hidden_size * num_heads, hidden_size)
        
        self.dropout = nn.Dropout(dropout)
        
        # STGNN specific utility functions
        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        
        self.scale = self.hidden_size ** 0.5

    def forward(self, x):
        pass